## EXAM SCHEDULER PROTOTYPE

In [52]:
import pandas as pd
from collections import defaultdict
import os

### CONSTANTS

In [53]:
SEM = '2024-2025-1'
DATA_PATH = './data/'

In [54]:
"""courses
This dictionary represents:
    key:    str     course_number
    value:  str     course_name
"""
courses: dict[str, str] = {}

# List all files in the specified directory
for filename in os.listdir(DATA_PATH):

    if filename.startswith(SEM) and not filename.endswith('report.csv'):
        # Construct the full path for the CSV file
        file_path = os.path.join(DATA_PATH, filename)
        
        # Load the relevant columns from the CSV file
        df = pd.read_csv(file_path, usecols=[0, 1], index_col=0)
        
        # Update the courses dictionary with course names
        courses.update(df['cname'].to_dict())

# At this point, the courses dictionary is populated

Printing all the courses that I am enrolled in

In [55]:
count = 0
for key, value in courses.items():
    if count > 5:
        break
    count += 1
    print(f"[{key}: {value}]")

[BIO101X: Biology I: Biomolecules]
[BIO103X: General Biology Laboratory]
[BIO201X: Biology III: Fundamentals of Molecular Biology]
[BIO203X: Biology V: Diversity of Life I]
[BIO205Y: Biology Laboratory I]
[BIO301N: Cell Biology]


In [56]:
"""course_students_dict
This dictionary represents:
    key:    str         course_name
    value:  set[int]    set of student IDs
"""
course_students_dict : dict[str, set] = defaultdict(set)


# Read the csv file `2024-2025-1_report.csv`
df = pd.read_csv(DATA_PATH + SEM + '_report.csv')

for index, row in df.iterrows():
    course_number = row['Course Number'].strip('"') 
    student_id = int(row['Roll Number'])           

    # Get course name; if not found, create a new entry
    course_name = courses.get(course_number)
    if course_name is None:
        course_name = row['Course Title'].strip('"')
        courses[course_number] = course_name

    # Add the student ID to the corresponding course name set
    course_students_dict[course_name].add(student_id)

`course_students_dict`: contains the mapping of course names to student IDs

In [57]:
import networkx as nx

In [58]:
g = nx.Graph()
g.add_edge('a', 'b')

In [59]:
g = nx.Graph()

for course in course_students_dict:
    for course2 in course_students_dict:

        if course == course2:
            continue

        if course in g and course2 in g[course]:
            continue

        if not course_students_dict[course].isdisjoint(course_students_dict[course2]):
            g.add_edge(course, course2)

In [60]:
colors = nx.coloring.greedy_color(g)

In [61]:
nx.coloring.greedy_color(g)

{'Law Relating to Intellectual Property and Patents': 0,
 'Reading Childrens Literature: Adventures, Ghosts and the Cultural Imagination': 1,
 'Data Science in Practice': 2,
 'Ethics for Research': 2,
 'MS Thesis': 0,
 'Learning, Memory, and Cognition': 3,
 'Ph.D. Thesis': 4,
 'Basic Organic Chemistry II': 4,
 'Natural Language Processing': 5,
 'Machine Learning': 3,
 'Real Analysis I': 4,
 'Waves and Optics': 5,
 'Fundamentals of Database Systems': 6,
 'Linear Algebra': 7,
 'Statistical Mechanics': 3,
 'Advances in Omics': 6,
 'Migration, State, and Society': 4,
 'Electromagnetism': 1,
 'Computer Vision': 8,
 'General Chemistry': 1,
 'Philosophical Beginnings: Matter, Motion, and the Cosmos': 7,
 'Introduction to Groups and Symmetry': 9,
 'Basic Electronics': 10,
 'Multivariable Calculus': 6,
 'Main Group Chemistry': 7,
 'Theory of Computation': 1,
 'Econometrics I': 2,
 'General Physics Laboratory I': 3,
 'Geohydrology': 8,
 'Basic Physical Chemistry': 8,
 'Physical Organic Chemistry

In [62]:
max_colours = max(colors.values())

print("Number of slots needed for exams: ", max_colours + 1)

Number of slots needed for exams:  18


### Visualizing the graph

In [63]:
import plotly.graph_objects as go

In [64]:
pos = nx.shell_layout(g)

fig = go.Figure()

for u, v, data in g.edges(data=True):
    x0, y0 = pos[u]
    x1, y1 = pos[v]
    fig.add_trace(go.Scatter(x=[x0, x1], y=[y0, y1], mode='lines'))

# Add nodes to the figure
for node in g.nodes():
    x, y = pos[node]
    fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers', marker=dict(size=10)))

# Show the figure
fig.show()